In [ ]:
%%capture
# !pip install datasets==2.6.1
!pip install install transformers==4.28
# !pip install huggingface_hub
!pip install torchaudio
# !pip install librosa
# !pip install jiwer

In [ ]:
pip install peft==0.2.0

In [ ]:
!pip install gradio

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install accelerate

In [ ]:
pip install bitsandbytes

In [ ]:
!pip install yt_dlp
import yt_dlp as youtube_dl

In [ ]:
import torch
import gradio as gr
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig

from transformers.pipelines.audio_utils import ffmpeg_read

import tempfile
import os


peft_model_id = "mfaiq2307/whisper-large-v2-indo-100steps"
language = "Indonesian"
task = "transcribe"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)
###coba Onnx
#model.eval()
#model.get_base_model().save_pretrained("./temp_lora_whisper_large_v2_mr")
#from optimum.onnxruntime import ORTModelForSpeechSeq2Seq
#
#model = ORTModelForSpeechSeq2Seq.from_pretrained(
#    "./temp_lora_whisper_large_v2_mr", from_transformers=True, provider="CUDAExecutionProvider"
#)
###coba ONNx
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
pipe = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor, chunk_length_s=30)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text

def download_yt_audio(yt_url, filename):
        info_loader = youtube_dl.YoutubeDL()
        try:
            info = info_loader.extract_info(yt_url, download=False)
        except youtube_dl.utils.DownloadError as err:
            raise gr.Error(str(err))
        ydl_opts = {"outtmpl": filename, "format": "worstvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"}
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            try:
                ydl.download([yt_url])
            except youtube_dl.utils.ExtractorError as err:
                raise gr.Error(str(err))

def transcribe_youtube(yt_url):
    with tempfile.TemporaryDirectory() as tmpdirname:
        filepath = os.path.join(tmpdirname, "video.mp4")
        download_yt_audio(yt_url, filepath)
        with open(filepath, "rb") as f:
            inputs = f.read()

    inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
    inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}

    with torch.cuda.amp.autocast():
        text = pipe(inputs, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]

    return text

audio_file = gr.Interface(
    fn=transcribe,
    inputs= gr.Audio(source="upload", type="filepath"),
    outputs="text",
    title="PEFT LoRA + INT8 Whisper Large V2 Indo",
    description="Realtime demo for Indonesian speech recognition using `PEFT-LoRA+INT8` fine-tuned Whisper Large V2 model.",
)

audio_mic = gr.Interface(
    fn=transcribe,
    inputs= gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="PEFT LoRA + INT8 Whisper Large V2 Indo",
    description="Realtime demo for Indonesian speech recognition using `PEFT-LoRA+INT8` fine-tuned Whisper Large V2 model.",
)

audio_yt = gr.Interface(
    fn=transcribe_youtube,
    inputs = gr.inputs.Textbox(lines=1, placeholder="Paste the URL to a YouTube video here", label="YouTube URL"),
    #inputs= gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="PEFT LoRA + INT8 Whisper Large V2 Indo",
    description="Realtime demo for Indonesian speech recognition using `PEFT-LoRA+INT8` fine-tuned Whisper Large V2 model.",
)

demo = gr.Blocks()

with demo:
    gr.TabbedInterface([audio_file, audio_mic, audio_yt], ["Audio File", "Microphone", "Youtube"])

demo.queue()
demo.launch(share=True)